In [47]:
#read and parse file, sort and return dict
def parse_log(file):
    log_dict = {}
    for log in file:
        elements = log.split()
        date = elements[0]
        time = elements[1]
        activity = elements[2]
        server = elements [3]
        email = elements[4]
        #add data to log_dict 
        if email in log_dict:
            if date in log_dict[email]:
                log_dict[email][date].append((time, activity, server))
            else:
                log_dict[email][date] = [(time, activity, server)]
        else:
            log_dict[email] = {date: [(time, activity, server)]}
        log_dict[email][date].sort()
    return log_dict

#read userlog.log
userlog = open("userlog.log")
file = userlog.readlines()
log_dict = parse_log(file)

In [48]:
#report suspicious activities
susAct = {}
susCount = 0
for email in log_dict:
    for date in log_dict[email]:
        loginCount = sum(1 for i in log_dict[email][date] if i[1] == 'login')
        lateLogin = any(i[1] == 'login' and int(i[0].split(':')[0]) < 5 for i in log_dict[email][date])
        
        if loginCount > 5 or lateLogin:
            susCount += 1
            if email in susAct:
                susAct[email].append(date)
            else:
                susAct[email] = [date]
            susAct[email].sort()

In [49]:
#report irresponsible behavior
irrBehav = {}
irrCount = 0
for email in log_dict:
    for date in log_dict[email]:
        loginCount = sum(1 for i in log_dict[email][date] if i[1] == 'login')
        logoutCount = sum(1 for i in log_dict[email][date] if i[1] == 'logout')
        
        if loginCount > logoutCount:
            irrCount += 1
            if email in irrBehav:
                irrBehav[email].append(date)
            else:
                irrBehav[email] = [date]
            irrBehav[email].sort()

In [50]:
#report system glitch
sysGlitch = {}
sysGlitchCount = 0
for email in log_dict:
    for date in log_dict[email]:
        logoutCount = sum(1 for i in log_dict[email][date] if i[1] == 'logout')
        loginCount = sum(1 for i in log_dict[email][date] if i[1] == 'login')
        
        if logoutCount > loginCount:
            sysGlitchCount += 1
            if email in sysGlitch:
                sysGlitch[email].append(date)
            else:
                sysGlitch[email] = [date]
            sysGlitch[email].sort()

In [51]:
#report domain count
emailList = log_dict.keys()
domain_dict = {}
for email in emailList:
    emailBit = email.split('@')
    domain = emailBit[1]
    if domain in domain_dict:
        domain_dict[domain] += 1
    else:
        domain_dict[domain] = 1
#create report for domain count
create_file = open("domain_report.txt", "x")
create_file.write("==============================\n")
create_file.write(f"=== Domain Count ({str(len(domain_dict))} Domains) ===\n")
create_file.write("==============================\n")
               
for domain in domain_dict:
    create_file.write(f"{domain}\t\t{domain_dict[domain]}\n")              
create_file.close()

In [52]:
#define the function to create reports for sus acts, irr behavior, and sys glitch
def writeReport(title, fileName, susAct, susCount, log_dict):
    create_file = open(fileName, "x")
    create_file.write("==============================\n")
    create_file.write(f"=== {title} ({str(susCount)} cases) ===\n")
    create_file.write("==============================\n")
    
    for email in susAct:
        countAct = len(susAct[email])
        create_file.write(f"{email}\t\t{str(countAct)}\n")
        
        for date in susAct[email]:
            create_file.write(f"\tDATE: [{date}] ---\n")
            
            for activity in log_dict[email][date]:
                create_file.write(f"\t\t{activity[0]}\t{activity[2]}\n")
                
    create_file.close()

In [53]:
#create reports for sus activities, irr behavior, and sys glitch
writeReport("Suspicious Activities", "suspicious_report.txt", susAct, susCount, log_dict)
writeReport("Irresponsible Behavior", "irresponsible_report.txt", irrBehav, irrCount, log_dict)
writeReport("System Glitch", "glitch_report.txt", sysGlitch, sysGlitchCount, log_dict)